In [1]:
import tensorflow as tf
print(tf.__version__)

2.2.0


In [2]:
import tensorflow.keras as tfk
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization, MaxPool2D, GlobalAveragePooling2D, Dense, Dropout, Activation
import pandas as pd
import numpy as np

In [3]:
data_dir = "../input/vietai-c6-assignment3-extracted-dataset/train.csv"
sub_dir = "../input/vietai-c6-assignment3-extracted-dataset/sample_submission.csv"
train_df = pd.read_csv(data_dir)
sub_df = pd.read_csv(sub_dir)

In [4]:
train_df.head()

,image,label
0,0.jpg,0
1,1.jpg,3
2,10.jpg,2
3,100.jpg,0
4,1000.jpg,0


In [5]:
sub_df.head()

,image,label
0,10010.jpg,0
1,10011.jpg,0
2,10028.jpg,0
3,10034.jpg,0
4,10056.jpg,0


In [6]:
classes = ["book", "can", "cardboard", "glass_bottle", "pen", "plastic_bottle"]
train_y = train_y = train_df.label
num_classes = len(np.unique(train_y))
y_ohe = tf.keras.utils.to_categorical(train_y, num_classes=num_classes)

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

size = 224
batch_size=32
train_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2, horizontal_flip=True, vertical_flip=True)
train_gen = train_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/train", batch_size=batch_size,
                                              target_size=(size, size), subset="training")

valid_gen = train_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/train", batch_size=batch_size,
                                          target_size=(size, size), subset="validation")

Found 25251 images belonging to 6 classes.
Found 6308 images belonging to 6 classes.


In [ ]:
def create_model():
    model = tfk.Sequential()
    model.add(Conv2D(144, (3, 3), strides=(1, 1), padding="valid", input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(94, (3, 3), strides=(1, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(144, (3, 3), strides=(1, 1), padding="same"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(94, (3, 3), strides=(1, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(Conv2D(42, (3, 3), strides=(1, 1), padding="valid"))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(MaxPool2D())
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dropout(0.21))
    model.add(Dense(256, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.21))
    model.add(Dense(256, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.21))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [8]:
def create_pretrained_model():
    model = tfk.Sequential()
    pretrained_net = tfk.applications.InceptionResNetV2(
        include_top=False,
        input_shape=(224, 224, 3),
        pooling="avg"
    )
    model.add(pretrained_net)
    model.add(Dropout(0.42))
    model.add(Dense(256, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.42))
    model.add(Dense(128, activation="tanh", kernel_regularizer=tf.keras.regularizers.l2()))
    model.add(Dropout(0.42))
    model.add(Dense(num_classes, activation="softmax"))
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    print(model.summary())
    return model

In [9]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if logs.get("val_accuracy") is not None:
            if(logs.get('val_accuracy') > 0.99):
                print("\nReached 99% val_accuracy so cancelling training!")
                self.model.stop_training = True
mcb = myCallback()

In [10]:
epochs = 150
use_efficientnet = True
if not use_efficientnet:
    model = create_model()
else:
    model = create_pretrained_model()

start_lr = 0.00001
min_lr = 0.00001
max_lr = 0.00005*8
rampup_epochs = 5
sustain_epochs = 0
exp_decay = .8

def lrfn(epoch):
    if epoch < rampup_epochs:
        return (max_lr - start_lr)/rampup_epochs * epoch + start_lr
    elif epoch < rampup_epochs + sustain_epochs:
        return max_lr
    else:
        return (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr

mcp = tf.keras.callbacks.ModelCheckpoint("my_model.h5", monitor="val_accuracy",
                        save_best_only=True, save_weights_only=True, period=3)
val_acc_earlyStop = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", 
                                                         patience = epochs//15, restore_best_weights = True)
lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)



219062272/219055592 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dropout (Dropout)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               393472    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_______________________________________________________

In [16]:
model.load_weights("../input/pretrained-inceptionres/my_model (1).h5")

In [29]:
model.evaluate(valid_gen)

198/198 [==============================] - 23s 116ms/step - loss: 0.1618 - accuracy: 0.9662


[0.16177572309970856, 0.9662333726882935]

In [35]:
train_data_gen = ImageDataGenerator(rescale=1./255, validation_split=0.2, horizontal_flip=True, vertical_flip=True)
train_gen = train_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/train", batch_size=batch_size,
                                              target_size=(size, size), subset="training", seed=42)

valid_gen = train_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/train", batch_size=batch_size,
                                          target_size=(size, size), subset="validation", seed=42)

Found 25251 images belonging to 6 classes.
Found 6308 images belonging to 6 classes.


In [36]:
model.fit(train_gen, validation_data=valid_gen, epochs=epochs,initial_epoch=45, callbacks=[mcp, mcb, val_acc_earlyStop, lr_callback])


Epoch 00046: LearningRateScheduler reducing learning rate to 1.0051839891835612e-05.
Epoch 46/150
790/790 [==============================] - 296s 375ms/step - loss: 0.0173 - accuracy: 0.9999 - val_loss: 0.1714 - val_accuracy: 0.9680 - lr: 1.0052e-05

Epoch 00047: LearningRateScheduler reducing learning rate to 1.004147191346849e-05.
Epoch 47/150
790/790 [==============================] - 297s 376ms/step - loss: 0.0184 - accuracy: 0.9996 - val_loss: 0.1723 - val_accuracy: 0.9673 - lr: 1.0041e-05

Epoch 00048: LearningRateScheduler reducing learning rate to 1.0033177530774792e-05.
Epoch 48/150
790/790 [==============================] - 297s 376ms/step - loss: 0.0168 - accuracy: 0.9999 - val_loss: 0.1838 - val_accuracy: 0.9650 - lr: 1.0033e-05

Epoch 00049: LearningRateScheduler reducing learning rate to 1.0026542024619834e-05.
Epoch 49/150
790/790 [==============================] - 296s 375ms/step - loss: 0.0167 - accuracy: 0.9999 - val_loss: 0.1736 - val_accuracy: 0.9681 - lr: 1.0027e-

In [43]:
model.save_weights("weight_model.h5")
model.save("full_model.h5")

In [41]:
model.evaluate(valid_gen)

198/198 [==============================] - 23s 115ms/step - loss: 0.1751 - accuracy: 0.9683


[0.17513923346996307, 0.9682942032814026]

In [42]:
test_data_gen = ImageDataGenerator(rescale=1.0/255)
test_generator = test_data_gen.flow_from_directory("../input/vietai-c6-assignment3-extracted-dataset/test",class_mode=None, target_size=(size, size), shuffle=False)
test_generator.reset()
pred = model.predict(test_generator)

# pred là một ma trận xác suất của ảnh trên các lớp.
# Ta lấy lớp có xác suất cao nhất trên từng ảnh bằng hàm argmax
pred_labels = np.argmax(pred, axis=1)
sub_df['label'] = pred_labels
sub_df.head(20)

Found 3837 images belonging to 1 classes.


,image,label
0,10010.jpg,2
1,10011.jpg,1
2,10028.jpg,2
3,10034.jpg,3
4,10056.jpg,5
5,10081.jpg,2
6,10084.jpg,1
7,10091.jpg,3
8,101.jpg,2
9,1010.jpg,5


In [34]:
sub_df.to_csv("submission.csv", index=False)